<a href="https://colab.research.google.com/github/kumar-abhishek/handson-ml2/blob/master/wip_normalized_padded_BachChorales_HandsOnChapter_15.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:

import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
import numpy as np


In [0]:
"""
X_train_000 = pd.read_csv('chorale_000.csv')
print(X_train_000.head(10))

X_valid_229 = pd.read_csv('chorale_229.csv')
print(X_valid_229.head(10))

X_train_000 = X_train_000.to_numpy()
X_valid_229 = X_valid_229.to_numpy()
"""
def data_create(X_train_000, X_valid_229):
  dataX=X_train_000[:-1]
  dataY=X_train_000[1:]
  dataValidationX=X_valid_229[:-1]
  dataValidationY=X_valid_229[1:]

  #dataValidationY=[]
  #for row in range(len(X_train_000)-1):
  #    dataX.append(X_train_000[row])
  #    dataY.append(X_train_000[row+1])
  print(dataX[0:10])
  print(dataY[0:10])
  print(type(dataX))

  #for row in range(len(X_valid_229)-1):
  #     dataValidationX.append(X_valid_229[row])
  #     dataValidationY.append(X_valid_229[row+1])
  print(dataValidationX[0:5])
  print(dataValidationY[0:5])
  return (dataX, dataY, dataValidationX, dataValidationY)

In [0]:
# normalization
from sklearn.preprocessing import MinMaxScaler
def normalize(dataX, dataY,dataValidationX, dataValidationY):
  scaler = MinMaxScaler( feature_range=(0, 1) )
  dataXScaler = scaler.fit_transform(dataX)
  dataYScaler = scaler.fit_transform(dataY)
  print(dataXScaler[0:10])
  print(dataYScaler[0:10])

  inversedX = scaler.inverse_transform(dataXScaler[0:10])
  print('inversed:')
  print(inversedX[0:10])
  print(scaler.inverse_transform(dataXScaler)[2:30])

  scalerValidation = MinMaxScaler( feature_range=(0, 1) )
  dataValidationX = scalerValidation.fit_transform(dataValidationX)
  dataValidationY = scalerValidation.fit_transform(dataValidationY)
  print(dataValidationX[0:5])
  print(dataValidationY[0:5])

  print(scaler.inverse_transform(dataXScaler)[2:30])
  return (dataXScaler, dataYScaler, dataValidationX, dataValidationY, scaler, scalerValidation)

In [21]:
from keras.preprocessing.sequence import pad_sequences
import math
# convert into input/output

def find_X_Y(dataXScaler, dataYScaler, dataValidationX, dataValidationY):
  i,j=0,0
  k,l=0,0
  X, y=[], []
  validX, validY=[], []
  #print(type(dataX))
  #print(type(dataX[0]), dataX[0])
  while i<len(dataXScaler):
    b=[]
    for item in range(0, i+1):
      b.append(dataXScaler[item])
    X.append(np.array(b))
    i += 1

  while j<len(dataYScaler):
    y.append(dataYScaler[j])
    j+= 1

  while k<len(dataValidationX):
    b=[]
    for item in range(0, k+1):
      b.append(dataValidationX[item])
    validX.append(np.array(b))
    k += 1


  # while k<len(dataValidationX):
  #   validX.append(np.array([dataValidationX[k]]))
  #   k += 1

  while l<len(dataValidationY):
    validY.append(dataValidationY[l])
    l+=1

  X=np.array(X)
  y=np.array(y)


  X=pad_sequences(X, padding='post', dtype='float32')
  #print(padded)

  print(X[0:5])
  print(y[4])
  print(X.shape, y.shape)


  validX=np.array(validX)
  validY=np.array(validY)
  validX=pad_sequences(validX, padding='post', dtype='float32')
  print(validX[0:2])
  print(validY[1])
  print(validX.shape, validY.shape)
  
  return (X, y, validX, validY)

Using TensorFlow backend.


In [0]:
from keras.models import Sequential

from keras import optimizers
from keras.layers import LSTM
from keras.layers import Dense, Dropout, BatchNormalization, TimeDistributed
# the dataset knows the number of features, e.g. 2

# fit model
def train_on_all_chorales(model, X, y, validX, validY):
  model.fit(X, y, epochs=200, validation_data=(validX, validY))

def train_model(X, y, validX, validY):
  n_features = 4

  # choose a number of time steps
  n_steps = None

  model = Sequential()
  #model.add(TimeDistributed(Dense(128), input_shape=(None, n_features)))
  model.add(LSTM(64, activation='tanh', input_shape=(None, n_features), return_sequences=True))
  model.add(BatchNormalization())

  model.add(LSTM(32 , activation = 'tanh'))
  model.add(BatchNormalization())
  model.add(Dropout(0.2))
  model.add(Dense(n_features, activation='softmax'))
  model.compile(optimizer='adam', loss='mse')
  train_on_all_chorales(model, X, y, validX, validY)
  return model


In [2]:
from google.colab import drive
import os
import numpy as np
import pandas as pd
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
def read_dir_of_chorales(dir):

  print("\nReading chorales in: " + dir + "\n---------------\n")

  chorales = []
  file_counter = 0
  for filename in sorted(os.listdir(dir)):
    file_counter += 1
    if (file_counter % 10) == 0:
      print(str(file_counter) + ") filename: " + filename)

    one_training_chorale = pd.read_csv(os.path.join(dir, filename)).to_numpy()
    chorales.append(one_training_chorale)

  return chorales

In [4]:
data_path = "/content/drive/My Drive/jsb_chorales"

train_path = os.path.join(data_path, 'train')
valid_path = os.path.join(data_path, 'valid')
test_path = os.path.join(data_path, 'test')

bach_training_chorales = read_dir_of_chorales(train_path)
bach_validation_chorales = read_dir_of_chorales(valid_path)
bach_test_chorales = read_dir_of_chorales(test_path)


Reading chorales in: /content/drive/My Drive/jsb_chorales/train
---------------

10) filename: chorale_009.csv
20) filename: chorale_019.csv
30) filename: chorale_029.csv
40) filename: chorale_039.csv
50) filename: chorale_049.csv
60) filename: chorale_059.csv
70) filename: chorale_069.csv
80) filename: chorale_079.csv
90) filename: chorale_089.csv
100) filename: chorale_099.csv
110) filename: chorale_109.csv
120) filename: chorale_119.csv
130) filename: chorale_129.csv
140) filename: chorale_139.csv
150) filename: chorale_149.csv
160) filename: chorale_159.csv
170) filename: chorale_169.csv
180) filename: chorale_179.csv
190) filename: chorale_189.csv
200) filename: chorale_199.csv
210) filename: chorale_209.csv
220) filename: chorale_219.csv

Reading chorales in: /content/drive/My Drive/jsb_chorales/valid
---------------

10) filename: chorale_238.csv
20) filename: chorale_248.csv
30) filename: chorale_258.csv
40) filename: chorale_268.csv
50) filename: chorale_278.csv
60) filename:

In [5]:
chorale_counter = 0
for chorale in bach_training_chorales:
  chorale_counter += 1
  if chorale_counter > 3: break

  print("Chorale #" + str(chorale_counter))
  print("Shape: " + str(chorale.shape))
  print(chorale[:6])
  print(" ...\n")

Chorale #1
Shape: (192, 4)
[[74 70 65 58]
 [74 70 65 58]
 [74 70 65 58]
 [74 70 65 58]
 [75 70 58 55]
 [75 70 58 55]]
 ...

Chorale #2
Shape: (228, 4)
[[69 64 61 57]
 [69 64 61 57]
 [69 64 61 57]
 [69 64 61 57]
 [71 64 59 56]
 [71 64 59 56]]
 ...

Chorale #3
Shape: (208, 4)
[[67 62 59 55]
 [67 62 59 55]
 [67 62 59 55]
 [67 62 59 55]
 [67 64 60 48]
 [67 64 60 48]]
 ...



**Q10[part-1]. Download the Bach chorales dataset and unzip it. It is composed of 382 chorales composed by Johann Sebastian Bach. Each chorale is 100 to 640 time steps long, and each time step contains 4 integers, where each integer corresponds to a note’s index on a piano (except for the value 0, which means that no note is played). Train a model—recurrent, convolutional, or both—that can predict the next time step (four notes), given a sequence of time steps from a chorale. **bold text**

In [48]:
#bach_validation_chorales
m = min(len(bach_training_chorales), len(bach_validation_chorales))
print(m)

for i in range(m):
  X_train = bach_training_chorales[i]
  X_valid = bach_validation_chorales[i]
  (dataX, dataY, dataValidationX, dataValidationY) = data_create(X_train, X_valid)
  (dataXScaler, dataYScaler, dataValidationXScaler, dataValidationYScaler, scaler, scalerValidation) = normalize(dataX, dataY, dataValidationX, dataValidationY)
  (X, y, validX, validY) = find_X_Y(dataXScaler, dataYScaler, dataValidationX, dataValidationY)
  model = train_model(X, y, validX, validY)

76
[[74 70 65 58]
 [74 70 65 58]
 [74 70 65 58]
 [74 70 65 58]
 [75 70 58 55]
 [75 70 58 55]
 [75 70 60 55]
 [75 70 60 55]
 [77 69 62 50]
 [77 69 62 50]]
[[74 70 65 58]
 [74 70 65 58]
 [74 70 65 58]
 [75 70 58 55]
 [75 70 58 55]
 [75 70 60 55]
 [75 70 60 55]
 [77 69 62 50]
 [77 69 62 50]
 [77 69 62 50]]
<class 'numpy.ndarray'>
[[72 67 60 48]
 [72 67 60 48]
 [72 67 60 48]
 [72 67 60 48]
 [72 67 64 48]]
[[72 67 60 48]
 [72 67 60 48]
 [72 67 60 48]
 [72 67 64 48]
 [72 67 64 48]]
[[0.44444444 0.77777778 1.         0.88235294]
 [0.44444444 0.77777778 1.         0.88235294]
 [0.44444444 0.77777778 1.         0.88235294]
 [0.44444444 0.77777778 1.         0.88235294]
 [0.55555556 0.77777778 0.41666667 0.70588235]
 [0.55555556 0.77777778 0.41666667 0.70588235]
 [0.55555556 0.77777778 0.58333333 0.70588235]
 [0.55555556 0.77777778 0.58333333 0.70588235]
 [0.77777778 0.66666667 0.75       0.41176471]
 [0.77777778 0.66666667 0.75       0.41176471]]
[[0.44444444 0.77777778 1.         0.88235294]
 

KeyboardInterrupt: ignored

In [37]:
# demonstrate prediction
x_input = X[12]
x_input = x_input.reshape((1, len(x_input), 4))
print(x_input)
yhat = model.predict(x_input, verbose=0)
#print(yhat)
print(scaler.inverse_transform(yhat))

print('expected: ', dataY[12])

[[[0.44444445 0.7777778  1.         0.88235295]
  [0.44444445 0.7777778  1.         0.88235295]
  [0.44444445 0.7777778  1.         0.88235295]
  [0.44444445 0.7777778  1.         0.88235295]
  [0.5555556  0.7777778  0.41666666 0.7058824 ]
  [0.5555556  0.7777778  0.41666666 0.7058824 ]
  [0.5555556  0.7777778  0.5833333  0.7058824 ]
  [0.5555556  0.7777778  0.5833333  0.7058824 ]
  [0.7777778  0.6666667  0.75       0.4117647 ]
  [0.7777778  0.6666667  0.75       0.4117647 ]
  [0.7777778  0.6666667  0.75       0.4117647 ]
  [0.7777778  0.6666667  0.75       0.4117647 ]
  [0.7777778  0.7777778  0.75       0.7058824 ]
  [0.         0.         0.         0.        ]
  [0.         0.         0.         0.        ]
  [0.         0.         0.         0.        ]
  [0.         0.         0.         0.        ]
  [0.         0.         0.         0.        ]
  [0.         0.         0.         0.        ]
  [0.         0.         0.         0.        ]
  [0.         0.         0.         0.  

In [41]:
# demonstrate prediction
print(scaler.inverse_transform(dataXScaler)[2:30])

x_input = dataXScaler[20:66]
x_input = x_input.reshape((1, len(x_input), 4))
print(scaler.inverse_transform(dataXScaler)[20:66])
yhat = model.predict(x_input, verbose=0)
print(scaler.inverse_transform(yhat))
print('expected: ', scaler.inverse_transform(dataYScaler)[66])

[[74. 70. 65. 58.]
 [74. 70. 65. 58.]
 [75. 70. 58. 55.]
 [75. 70. 58. 55.]
 [75. 70. 60. 55.]
 [75. 70. 60. 55.]
 [77. 69. 62. 50.]
 [77. 69. 62. 50.]
 [77. 69. 62. 50.]
 [77. 69. 62. 50.]
 [77. 70. 62. 55.]
 [77. 70. 62. 55.]
 [77. 69. 62. 55.]
 [77. 69. 62. 55.]
 [75. 67. 63. 48.]
 [75. 67. 63. 48.]
 [75. 69. 63. 48.]
 [75. 69. 63. 48.]
 [74. 70. 65. 46.]
 [74. 70. 65. 46.]
 [74. 70. 65. 46.]
 [74. 70. 65. 46.]
 [72. 69. 65. 53.]
 [72. 69. 65. 53.]
 [72. 69. 65. 53.]
 [72. 69. 65. 53.]
 [72. 69. 65. 53.]
 [72. 69. 65. 53.]]
[[74. 70. 65. 46.]
 [74. 70. 65. 46.]
 [74. 70. 65. 46.]
 [74. 70. 65. 46.]
 [72. 69. 65. 53.]
 [72. 69. 65. 53.]
 [72. 69. 65. 53.]
 [72. 69. 65. 53.]
 [72. 69. 65. 53.]
 [72. 69. 65. 53.]
 [72. 69. 65. 53.]
 [72. 69. 65. 53.]
 [74. 70. 65. 46.]
 [74. 70. 65. 46.]
 [74. 70. 65. 46.]
 [74. 70. 65. 46.]
 [75. 69. 63. 48.]
 [75. 69. 63. 48.]
 [75. 67. 63. 48.]
 [75. 67. 63. 48.]
 [77. 65. 62. 50.]
 [77. 65. 62. 50.]
 [77. 65. 60. 50.]
 [77. 65. 60. 50.]
 [74. 67. 5

**Q10[part-2] Then use this model to generate Bach-like music, one note at a time: you can do this by giving the model the start of a chorale and asking it to predict the next time step, then appending these time steps to the input sequence and asking the model for the next note, and so on. Also make sure to check out Google’s Coconet model, which was used for a nice Google doodle about Bach.**

In [40]:

# choose a number of time steps
n_steps = None
# convert into input/output
i=1
n=1
x_input = np.array([dataXScaler[0]])

x_input = x_input.reshape((1, len(x_input), n_features))
while i<len(dataXScaler):
  # demonstrate prediction
  print('Input: ')
  for input in x_input:
    print(scaler.inverse_transform(input))
  print('---------------')
  yhat = model.predict(x_input, verbose=1)
  print('Predicted Output: ', scaler.inverse_transform(yhat))
  print('expected: ', scaler.inverse_transform(dataYScaler)[i])
  print('\n\n')

  yhat = yhat.reshape((1, len(yhat), n_features))
  x_input = np.concatenate([x_input, yhat], axis=1)
  i += 1
  if i>20:
    break

Input: 
[[74. 70. 65. 58.]]
---------------
1/1 [==============================] - 0s 2ms/step
Predicted Output:  [[70.00305  63.       64.99538  43.000782]]
expected:  [74. 70. 65. 58.]



Input: 
[[74.         70.         65.         58.        ]
 [70.00305222 63.00000001 64.99537802 43.00078277]]
---------------
1/1 [==============================] - 0s 2ms/step
Predicted Output:  [[70.007996 63.       64.97847  43.015408]]
expected:  [74. 70. 65. 58.]



Input: 
[[74.         70.         65.         58.        ]
 [70.00305222 63.00000001 64.99537802 43.00078277]
 [70.00799361 63.00000214 64.97846508 43.01540476]]
---------------
1/1 [==============================] - 0s 1ms/step
Predicted Output:  [[70.06947  63.001442 64.371254 43.75677 ]]
expected:  [75. 70. 58. 55.]



Input: 
[[74.         70.         65.         58.        ]
 [70.00305222 63.00000001 64.99537802 43.00078277]
 [70.00799361 63.00000214 64.97846508 43.01540476]
 [70.06947329 63.00144369 64.3712523  43.7567717 ]]


In [0]:
# Using SimpleRNN 
print(n_features)
model_rnn = keras.models.Sequential([
    keras.layers.SimpleRNN(100, return_sequences=True, input_shape=[None, n_features]),
    keras.layers.SimpleRNN(100),
    keras.layers.Dense(n_features)
])

model_rnn.compile(loss="mse", optimizer="adam")
print(X.shape, y.shape)
history = model_rnn.fit(X, y, epochs=500,
                    validation_data=(validX, validY))

4
(191, 191, 4) (191, 4)
Train on 191 samples, validate on 195 samples
Epoch 1/500
191/191 [==============================] - 3s 17ms/sample - loss: 3884.6061 - val_loss: 3738.2476
Epoch 2/500
191/191 [==============================] - 1s 4ms/sample - loss: 3597.8243 - val_loss: 3501.6842
Epoch 3/500
191/191 [==============================] - 1s 4ms/sample - loss: 3393.6590 - val_loss: 3344.9362
Epoch 4/500
191/191 [==============================] - 1s 4ms/sample - loss: 3272.6073 - val_loss: 3257.6440
Epoch 5/500
191/191 [==============================] - 1s 4ms/sample - loss: 3191.5464 - val_loss: 3182.6286
Epoch 6/500
191/191 [==============================] - 1s 4ms/sample - loss: 3118.7115 - val_loss: 3110.6376
Epoch 7/500
191/191 [==============================] - 1s 4ms/sample - loss: 3049.0692 - val_loss: 3040.0778
Epoch 8/500
191/191 [==============================] - 1s 4ms/sample - loss: 2978.6175 - val_loss: 2971.6227
Epoch 9/500
191/191 [==============================] - 1

In [0]:
# choose a number of time steps
n_steps = None
# convert into input/output
i=1
n=1
#x_input = X[0]
x_input = np.array([dataX[0]])

x_input = x_input.reshape((1, len(x_input), n_features))
while i<len(X):
  # demonstrate prediction
  #x_input = x_input.reshape((1, len(x_input), n_features))
  print('Input: ', x_input)
  print('---------------')
  print('Expected Output: ', y[i])
  print('---------------')
  yhat = model_rnn.predict(x_input, verbose=1)
  print('Predicted Output: ', yhat)
  print("\n\n")
  yhat = yhat.reshape((1, len(yhat), n_features))
  x_input = np.concatenate([x_input, yhat], axis=1)
  i += 1
  if i>20:
    break

Input:  [[[74 70 65 58]]]
---------------
Expected Output:  [74 70 65 58]
---------------
1/1 [==============================] - 0s 3ms/sample
Predicted Output:  [[72.12551  65.19431  58.163357 50.04461 ]]



Input:  [[[74.         70.         65.         58.        ]
  [72.12551117 65.19431305 58.16335678 50.04460907]]]
---------------
Expected Output:  [74 70 65 58]
---------------
1/1 [==============================] - 0s 2ms/sample
Predicted Output:  [[74.03032  67.032486 59.8005   51.336597]]



Input:  [[[74.         70.         65.         58.        ]
  [72.12551117 65.19431305 58.16335678 50.04460907]
  [74.03031921 67.03248596 59.80049896 51.33659744]]]
---------------
Expected Output:  [75 70 58 55]
---------------
1/1 [==============================] - 0s 2ms/sample
Predicted Output:  [[74.03046  67.032616 59.80062  51.336697]]



Input:  [[[74.         70.         65.         58.        ]
  [72.12551117 65.19431305 58.16335678 50.04460907]
  [74.03031921 67.03248596 59.800

In [0]:

# using GRU
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
# the dataset knows the number of features, e.g. 2
n_features = X.shape[2]
print(n_features)

# choose a number of time steps
n_steps = None

model_gru = keras.models.Sequential([
    keras.layers.GRU(100, return_sequences=True, input_shape=[n_steps, n_features]),
    keras.layers.GRU(100),
    keras.layers.Dense(n_features)
])
model_gru.compile(optimizer='adam', loss='mse')
# fit model
model_gru.fit(X, y, epochs=250, validation_data=(validX, validY))


4
Train on 191 samples, validate on 195 samples
Epoch 1/250
191/191 [==============================] - 7s 38ms/sample - loss: 3954.2694 - val_loss: 3808.7841
Epoch 2/250
191/191 [==============================] - 4s 20ms/sample - loss: 3655.6493 - val_loss: 3534.8152
Epoch 3/250
191/191 [==============================] - 4s 20ms/sample - loss: 3411.2531 - val_loss: 3333.9724
Epoch 4/250
191/191 [==============================] - 4s 20ms/sample - loss: 3238.6062 - val_loss: 3211.2050
Epoch 5/250
191/191 [==============================] - 4s 20ms/sample - loss: 3144.1109 - val_loss: 3131.7926
Epoch 6/250
191/191 [==============================] - 4s 21ms/sample - loss: 3067.1131 - val_loss: 3056.0901
Epoch 7/250
191/191 [==============================] - 4s 21ms/sample - loss: 2992.6149 - val_loss: 2982.3321
Epoch 8/250
191/191 [==============================] - 4s 21ms/sample - loss: 2920.1653 - val_loss: 2910.4434
Epoch 9/250
191/191 [==============================] - 4s 20ms/sample - 

KeyboardInterrupt: ignored

In [0]:
# demonstrate prediction
x_input = X[0:5]
x_input = x_input.reshape((1, len(x_input), n_features))
print(x_input)
yhat = model_gru.predict(x_input, verbose=0)
print(yhat)

[[[74 70 65 58]
  [74 70 65 58]
  [74 70 65 58]
  [74 70 65 58]
  [75 70 58 55]]]
[[73.542854 66.80782  59.76219  51.33617 ]]


In [0]:
# choose a number of time steps
n_steps = None
# convert into input/output
i=1
n=1
#x_input = X[0]
x_input = np.array([dataX[0]])
x_input = x_input.reshape((1, len(x_input), n_features))
while i<len(X):
  # demonstrate prediction
  print('Input: ', x_input)
  print('---------------')
  print('Expected Output: ', y[i])
  print('---------------')
  yhat = model_gru.predict(x_input, verbose=1)
  print('Predicated Output: ', yhat)
  print("\n\n")
  yhat = yhat.reshape((1, len(yhat), n_features))
  x_input = np.concatenate([x_input, yhat], axis=1)
  i += 1
  if i>10:
    break

Input:  [[[74 70 65 58]]]
---------------
Expected Output:  [74 70 65 58]
---------------
1/1 [==============================] - 0s 5ms/sample
Predicated Output:  [[73.5445   66.80927  59.76339  51.336994]]



Input:  [[[74.         70.         65.         58.        ]
  [73.54450226 66.80927277 59.76338959 51.33699417]]]
---------------
Expected Output:  [74 70 65 58]
---------------
1/1 [==============================] - 0s 2ms/sample
Predicated Output:  [[73.54287  66.80782  59.762207 51.33618 ]]



Input:  [[[74.         70.         65.         58.        ]
  [73.54450226 66.80927277 59.76338959 51.33699417]
  [73.54286957 66.80782318 59.76220703 51.33618164]]]
---------------
Expected Output:  [75 70 58 55]
---------------
1/1 [==============================] - 0s 2ms/sample
Predicated Output:  [[73.542854 66.807816 59.76219  51.336174]]



Input:  [[[74.         70.         65.         58.        ]
  [73.54450226 66.80927277 59.76338959 51.33699417]
  [73.54286957 66.80782318 59.